In [ ]:
#pip install gspread oauth2client
#pip install quopri
#pip install beautifulsoup4

# Importing libraries

import imaplib, email
import quopri
from bs4 import BeautifulSoup
import requests
import datetime
import gspread
import time
import pyautogui
import glob
import os
import webbrowser
from oauth2client.service_account import ServiceAccountCredentials

In [ ]:
#Necessary Credentials
user = 'dummy2456542@gmail.com'
password = 'jjaqkopmjbhbmnit'
imap_url = 'imap.gmail.com'

#Chrome path to Start Chrome and automate tab opening process
chrome_path = 'C:/Program Files (x86)/Google/Chrome/Application/chrome.exe %s'

def send_sms(mob,key):
    '''Connect Some SMS API here'''


def update_google_sheets(cust_dict):
    
    scope = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]
    
    # Assign credentials and path of style sheet
    creds = ServiceAccountCredentials.from_json_keyfile_name("charming-mile-354117-9b5fd26ced79.json", scope)
    client = gspread.authorize(creds)
    sheet = client.open("Netlux").sheet1
    
    #Google Sheet Update Operations
    sheet.update_cell(len(sheet.col_values(1))+1, 1, str(datetime.datetime.now()))
    sheet.update_cell(len(sheet.col_values(3))+1, 3, cust_dict['Name'])
    sheet.update_cell(len(sheet.col_values(4))+1, 4, cust_dict['Mobile'])
    if cust_dict['Payment']=='PAID':
        sheet.update_cell(len(sheet.col_values(1)), 5, 'done')
        send_sms(cust_dict['Mobile'],sheet.cell(len(sheet.col_values(1)), 2).value)


def get_links(txt):
    
    #To extract links from href of <a href=""></a>
    soup = BeautifulSoup(quopri.decodestring(txt), "html.parser")
    links = soup.find_all('a')
    list_of_links=list()
    for tag in links:
        link = tag.get('href',None)
        if link is not None:
            list_of_links.append(link)
            
    #Return required link
    return list_of_links[2]
    
def parse_html(link):
    
    #Open the link extracted from Mail
    webbrowser.get(chrome_path).open(link)
    time.sleep(10)
    
    #Simulate Ctrl+S
    pyautogui.keyDown('ctrl') # hold ctrl key
    pyautogui.press('s') # press s key
    pyautogui.keyUp('ctrl') # release ctrl key
    time.sleep(2)
    pyautogui.press('enter')
    time.sleep(10)
    
    all_htmls = glob.glob('C:\\Users\\Asus\\Downloads\\*.html') # *html consider only html files in mentioned path
    latest_file = max(all_htmls, key=os.path.getctime)          # Open latest html file
    HTMLFile = open(latest_file, "r") 
    index = HTMLFile.read()
    HTMLFile.close()
    #Parse the contents of html file to find the required data
    soup = BeautifulSoup(index, "html.parser")
    span_text = soup.find_all('span')
    cust_details=list()
    for item in span_text:
        cust_details.append(item.get_text())
    cust_name=cust_details[17]
    cust_contact=cust_details[18]
    div_text = soup.find_all('div')
    cust_details=list()
    for item in div_text:
        cust_details.append(item.get_text())
    cust_payment=cust_details[105]
    return {'Name':cust_name,'Mobile':cust_contact,'Payment':cust_payment}

# Function to get email content part i.e its body part
def get_body(msg):
    if msg.is_multipart():
        return get_body(msg.get_payload(0))
    else:
        return msg.get_payload(None, True)

# Function to search for a key value pair
def search(key, value, con):
    result, data = con.search(None, key, '"{}"'.format(value),'UnSeen')
    return data

# Function to get the list of emails under this label
def get_emails(result_bytes, con):
    
    msgs = [] # all the email data are pushed into a list
    for num in result_bytes[0].split():
        typ, data = con.fetch(num, '(RFC822)')
        msgs.append(data)

    return msgs

def automate(user,password,imap_url):
    
    # this is done to make SSL connection with GMAIL
    con = imaplib.IMAP4_SSL(imap_url)

    # logging the user in
    con.login(user, password)

    # calling function to check for email under this label
    con.select('Inbox')

    # fetching emails from this user "verma.aarohan@gmail.com"
    msgs = get_emails(search('FROM', 'verma.aarohan@gmail.com', con),con) #Mail Subject Emoji Issue
                                                                        #'(SUBJECT "Congratulations , you have a new order")'
    # Finding the required content from our msgs
    
    for msg in msgs[::-1]:
        for sent in msg:
            if type(sent) is tuple: 
                link=get_links(str(sent[1],'utf-8'))
                cust_dict=parse_html(link)
                update_google_sheets(cust_dict)
                time.sleep(10)
        time.sleep(10)
    con.close()
    con.logout()

while True:
    automate(user,password,imap_url)
    time.sleep(600)

# To Demonstrate Automation of Google Sheets

In [1]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import datetime

scope = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]
# Assign credentials ann path of style sheet
creds = ServiceAccountCredentials.from_json_keyfile_name("charming-mile-354117-9b5fd26ced79.json", scope)
client = gspread.authorize(creds)
sheet = client.open("Netlux").sheet1
sheet.update_cell(len(sheet.col_values(1))+1, 1, str(datetime.datetime.now()))

{'spreadsheetId': '1tDn_6tYrkDVY7p62uf9vNdyPqdQIwnrC6msq2C1oJ98',
 'updatedRange': 'Sheet1!A2',
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

# To Demonstrate Parsing of HTML File

In [2]:
from bs4 import BeautifulSoup
HTMLFile = open("Dukaan _ #3835300 order details.html", "r") 
index = HTMLFile.read()
HTMLFile.close()
soup = BeautifulSoup(index, "html.parser")
span_text = soup.find_all('span')
cust_details=list()
for item in span_text:
    cust_details.append(item.get_text())
print(cust_details[17])
print(cust_details[18])
div_text = soup.find_all('div')
cust_details=list()
for item in div_text:
    cust_details.append(item.get_text())
print(cust_details[105])

Bibhuti
+91-9556983698
PAID
